In [1]:
import pandas as pd
import pickle
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense , Input , LSTM , Embedding, Dropout , Activation, GRU, Flatten,Conv2D,Conv1D,MaxPooling1D, Dropout
from keras.layers import concatenate
from keras.layers import Bidirectional, GlobalMaxPool1D,BatchNormalization
from keras.models import Model, Sequential
from keras.layers import Convolution1D,GlobalMaxPooling1D
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.utils import to_categorical
import numpy as np
import sys
import os
import tensorflow as tf
import keras.backend.tensorflow_backend as ktf

Using TensorFlow backend.


In [2]:
#######################################################################
# Read in Data and tokenize , prepare training data and test data
#df = pd.read_csv("C:/Users/Harvey/Desktop/Yelp_data_set/restuarant_review_5_label_unbalanced.csv")
#df = pd.read_csv("/home/ec2-user/Data/restuarant_review_5_label_unbalanced.csv")



#df = pd.read_csv("C:/Users/Harvey/Desktop/Yelp_data_set/restuarant_review_balanced.csv"
#df = pd.read_csv("/home/ec2-user/Data/restuarant_review_5_label_unbalanced.csv")
#df = pd.read_csv("/usr4/cs542sp/zzjiang/Data/restuarant_review_5_label_unbalanced.csv")
#train= pd.read_csv("/Users/harvey/Desktop/Data/restuarant_balanced_2_train.csv",lineterminator='\n')
#test = pd.read_csv("/Users/harvey/Desktop/Data/restuarant_balanced_2_test.csv",lineterminator='\n')

train= pd.read_csv("/home/ec2-user/Data/restuarant_balanced_2_train.csv",lineterminator='\n')
test = pd.read_csv("/home/ec2-user/Data/restuarant_balanced_2_test.csv",lineterminator='\n')
print(train.shape)
print(test.shape)
print(np.unique(train['stars']))
print(np.unique(test['stars']))

(208160, 2)
(50000, 2)
[0 1]
[0 1]


In [12]:
maxlen = 130
max_features = 6000
tokenizer = Tokenizer(num_words=max_features)

tokenizer.fit_on_texts(train['Processed_Reviews'])
list_tokenized_train = tokenizer.texts_to_sequences(train['Processed_Reviews'])
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
y =train['stars']
#####################
# Test data
tokenizer.fit_on_texts(test['Processed_Reviews'])
list_tokenized_test = tokenizer.texts_to_sequences(test['Processed_Reviews'])
X_test = pad_sequences(list_tokenized_test, maxlen=maxlen)
y_test = test['stars']

In [4]:
#####################################################################
# Using pretrained glove vector
#####################################################################
#GLOVE_DIR = "/usr4/cs542sp/zzjiang/Data/"
GLOVE_DIR ="/home/ec2-user/Data/"
#GLOVE_DIR = "/Users/harvey/Desktop/Data/"
embeddings_index = {}
f = open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'),encoding = 'utf8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Total %s word vectors in Glove 6B 100d.' % len(embeddings_index))


word_index = tokenizer.word_index
embed_size = 100 
embedding_matrix = np.random.random((len(word_index) + 1, embed_size))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

      
embedding_layer = Embedding(len(word_index) + 1,
                            embed_size,
                            weights=[embedding_matrix],
                            input_length=maxlen,
                            trainable=True)

#Randomly initialized 
#embedding_layer = Embedding(len(word_index) + 1,
#                            embed_size,
#                            input_length=maxlen,
#                            trainable=False)




Total 400000 word vectors in Glove 6B 100d.


In [27]:
#############################################
# Original yoon kim with batch norm and drop out0
#############################################
conv_filters = 128
drop_out_rate = 0.1 + np.random.rand()*0.25
sequence_input = Input(shape=(maxlen,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)

# Specify each convolution layer and their kernel siz i.e. n-grams 
conv1_1 = Conv1D(filters=conv_filters, kernel_size=2,kernel_regularizer=regularizers.l2(0.01))(embedded_sequences)
btch1_1 = BatchNormalization()(conv1_1)
actv1_1 = Activation('relu')(btch1_1)
conv1_2 = Conv1D(filters=conv_filters, kernel_size=2,kernel_regularizer=regularizers.l2(0.01))(actv1_1)
btch1_2 = BatchNormalization()(conv1_2)
actv1_2 = Activation('relu')(btch1_2)
glmp1_1 = MaxPooling1D(pool_size = 4)(actv1_2)


conv2_1 = Conv1D(filters=conv_filters, kernel_size=3,kernel_regularizer=regularizers.l2(0.01))(embedded_sequences)
btch2_1 = BatchNormalization()(conv2_1)
actv2_1 = Activation('relu')(btch2_1)
conv2_2 = Conv1D(filters=conv_filters, kernel_size=3,kernel_regularizer=regularizers.l2(0.01))(actv2_1)
btch2_2 = BatchNormalization()(conv2_2)
actv2_2 = Activation('relu')(btch2_2)
glmp2_1 = MaxPooling1D(pool_size = 4)(actv2_2)




conv3_1 = Conv1D(filters=conv_filters, kernel_size=4,kernel_regularizer=regularizers.l2(0.01))(embedded_sequences)
btch3_1 = BatchNormalization()(conv3_1)
actv3_1 = Activation('relu')(btch3_1)
conv3_2 = Conv1D(filters=conv_filters, kernel_size=4,kernel_regularizer=regularizers.l2(0.01))(actv3_1)
btch3_2 = BatchNormalization()(conv3_2)
actv3_2 = Activation('relu')(btch3_2)
glmp3_1 = MaxPooling1D(pool_size = 4)(actv3_2)


conv4_1 = Conv1D(filters=conv_filters, kernel_size=5,kernel_regularizer=regularizers.l2(0.01))(embedded_sequences)
btch4_1 = BatchNormalization()(conv4_1)
actv4_1 = Activation('relu')(btch4_1)
conv4_2 = Conv1D(filters=conv_filters, kernel_size=5,kernel_regularizer=regularizers.l2(0.01))(actv4_1)
btch4_2 = BatchNormalization()(conv4_2)
actv4_2 = Activation('relu')(btch4_2)
glmp4_1 = MaxPooling1D(pool_size = 4)(actv4_2)

# Gather all convolution layers
cnct = concatenate([glmp1_1, glmp2_1, glmp3_1, glmp4_1], axis=1)
drp = Dropout(drop_out_rate)(cnct)

dns1  = Dense(256, activation='relu',kernel_regularizer=regularizers.l2(0.01))(drp)
btch1 = BatchNormalization()(dns1)
drp1  = Dropout(drop_out_rate)(btch1)
dns2  = Dense(128, activation='relu',kernel_regularizer=regularizers.l2(0.01))(drp1)
btch2 = BatchNormalization()(dns2)
drp2 = Dropout(drop_out_rate)(btch2)
flat = Flatten()(drp2)
out = Dense(1, activation='sigmoid')(flat)


model = Model(inputs=sequence_input, outputs=out)
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
model.summary()



__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            (None, 130)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 130, 100)     16280600    input_8[0][0]                    
__________________________________________________________________________________________________
conv1d_49 (Conv1D)              (None, 129, 128)     25728       embedding_1[7][0]                
__________________________________________________________________________________________________
conv1d_51 (Conv1D)              (None, 128, 128)     38528       embedding_1[7][0]                
__________________________________________________________________________________________________
conv1d_53 

In [ ]:
batch_size = 512
epochs = 20
history = model.fit(X_t,y, batch_size=batch_size, epochs=epochs, validation_split=0.2)

Train on 166528 samples, validate on 41632 samples
Epoch 1/20
166528/166528 [==============================] - 116s 696us/step - loss: 2.4909 - acc: 0.9356 - val_loss: 0.4935 - val_acc: 0.8171
Epoch 2/20
166528/166528 [==============================] - 108s 651us/step - loss: 0.2075 - acc: 0.9565 - val_loss: 0.7579 - val_acc: 0.8494
Epoch 3/20
166528/166528 [==============================] - 108s 652us/step - loss: 0.1722 - acc: 0.9606 - val_loss: 0.4748 - val_acc: 0.9038
Epoch 4/20
166528/166528 [==============================] - 108s 651us/step - loss: 0.1566 - acc: 0.9636 - val_loss: 0.5659 - val_acc: 0.8559
Epoch 5/20
166528/166528 [==============================] - 108s 649us/step - loss: 0.1450 - acc: 0.9657 - val_loss: 1.9755 - val_acc: 0.6982
Epoch 6/20
166528/166528 [==============================] - 109s 652us/step - loss: 0.1392 - acc: 0.9669 - val_loss: 2.0334 - val_acc: 0.7508
Epoch 7/20
 81920/166528 [=============>................] - ETA: 51s - loss: 0.1353 - acc: 0.9682

In [ ]:
prediction = model.predict(X_test)
y_pred = (prediction > 0.5).reshape(-1,)
print(y_pred[:10])
print(y_test.shape)
print(y_pred.shape)
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score, confusion_matrix
print('accuracy :{0}'.format(accuracy_score(y_pred, y_test)))

In [ ]:
#################################################################
#Save train history as dict 
#################################################################

with open(r"/usr4/cs542sp/zzjiang/History/Yoon_kim_original", "wb") as output_file:
    pickle.dump(history.history, output_file)


